<a href="https://colab.research.google.com/github/heartbeatmaker/gan-face-generator/blob/master/face_generator_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import datetime

Using TensorFlow backend.


In [6]:
# Run this for Google CoLab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Generation resolution - Must be square 
# Training data is also scaled to this.
# Note GENERATE_RES higher than 4 will blow Google CoLab's memory.
GENERATE_RES = 2 # (1=32, 2=64, 3=96, etc.)
GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
IMAGE_CHANNELS = 3

# Preview image 
PREVIEW_ROWS = 1 # you can set the number of pictures
PREVIEW_COLS = 1
PREVIEW_MARGIN = 0  # you can set the margin between pictures
SAVE_FREQ = 100

# Size vector to generate images from
SEED_SIZE = 100

# Configuration
DATA_PATH = '/content/drive/My Drive/projects/faces'
EPOCHS = 10000
BATCH_SIZE = 32

print(f"Will generate {GENERATE_SQUARE}px square images.")

Will generate 96px square images.


In [10]:
generator = load_model(os.path.join(DATA_PATH,'face_generator.h5'))

noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, SEED_SIZE))


image_array = np.full((
    PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)),
    PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3),
    255, dtype=np.uint8)

generated_images = generator.predict(noise)

generated_images = 0.5 * generated_images + 0.5

image_count = 0
for row in range(PREVIEW_ROWS):
    for col in range(PREVIEW_COLS):
        r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] = generated_images[image_count] * 255
        image_count += 1


output_path = os.path.join(DATA_PATH,'results')
if not os.path.exists(output_path):
    os.makedirs(output_path)

basename = "generated_face"
suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
extension = ".png"
filename = basename+"_"+suffix+extension
    
filepath = os.path.join(output_path,filename)
im = Image.fromarray(image_array)
im.save(filepath)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


ValueError: ignored